Final Dataset =
- COCO (vehicles only) : https://cocodataset.org/#download
- UAVDT : https://sites.google.com/view/grli-uavdt

~/datasets/
 └── coco2017/
      ├── train2017/
      ├── val2017/
      └── annotations/
           ├── instances_train2017.json
           └── instances_val2017.json

In [15]:
# pip install ultralytics
#!pip install -U ultralytics
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco

convert_coco(
    labels_dir="datasets/coco2017/annotations",
    save_dir="datasets/coco_vehicle_yolo",
)

# coco_vehicle_yolo/
#  ├── images/
#  │    ├── train2017/
#  │    └── val2017/
#  └── labels/
#       ├── train2017/
#       └── val2017/

Annotations /Users/harshdeepkalita/projects/yolo26/datasets/coco2017/annotations/instances_train2017.json: 100% ━━━━━━━━━━━━ 117266/117266 8.4Kit/s 14.0s0.1s
Annotations /Users/harshdeepkalita/projects/yolo26/datasets/coco2017/annotations/instances_val2017.json: 100% ━━━━━━━━━━━━ 4952/4952 9.8Kit/s 0.5s0.0s
COCO data converted successfully.
Results saved to /Users/harshdeepkalita/projects/yolo26/datasets/coco_vehicle_yolo


In [18]:
import os
import shutil

src_root = "datasets/coco_vehicle_yolo"
dst_root = "datasets/coco_vehicle_only"

vehicle_classes = [2, 5, 7]  # COCO YOLO IDs

for split in ["train2017", "val2017"]:

    src_img_dir = os.path.join(src_root, "images", split)
    src_lbl_dir = os.path.join(src_root, "labels", split)

    dst_img_dir = os.path.join(dst_root, "images", split)
    dst_lbl_dir = os.path.join(dst_root, "labels", split)

    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    for label_file in os.listdir(src_lbl_dir):

        src_label_path = os.path.join(src_lbl_dir, label_file)

        with open(src_label_path, "r") as f:
            lines = f.readlines()

        filtered_lines = []

        for line in lines:
            cls_id = int(line.split()[0])
            if cls_id in vehicle_classes:
                filtered_lines.append(line)

        if len(filtered_lines) > 0:
            img_name = label_file.replace(".txt", ".jpg")
            src_img_path = os.path.join(src_img_dir, img_name)

            if os.path.exists(src_img_path):
                shutil.copy(src_img_path, os.path.join(dst_img_dir, img_name))

                with open(os.path.join(dst_lbl_dir, label_file), "w") as f:
                    f.writelines(filtered_lines)

print("✅ COCO filtered to vehicle-only labels (no remapping).")

✅ COCO filtered to vehicle-only labels (no remapping).


In [11]:
import os
import cv2
import random
import shutil

uavdt_img_root = "datasets/UAV-benchmark-M"
uavdt_gt_root = "datasets/UAV-benchmark-MOTD_v1.0/GT"
output_root = "datasets/vehicles"

train_img_dir = os.path.join(output_root, "images/train")
val_img_dir = os.path.join(output_root, "images/val")
train_lbl_dir = os.path.join(output_root, "labels/train")
val_lbl_dir = os.path.join(output_root, "labels/val")

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

train_ratio = 0.8

all_sequences = sorted(os.listdir(uavdt_img_root))

for seq in all_sequences:
    seq_path = os.path.join(uavdt_img_root, seq)
    gt_file = os.path.join(uavdt_gt_root, f"{seq}_gt.txt")

    if not os.path.exists(gt_file):
        continue

    # Read annotations
    annotations = {}
    with open(gt_file, "r") as f:
        for line in f:
            parts = line.strip().split(",")

            frame_id = int(parts[0])
            x = float(parts[2])
            y = float(parts[3])
            w = float(parts[4])
            h = float(parts[5])
            category = int(parts[7])

            # UAVDT categories:
            # 1 = car
            # 2 = truck
            # 3 = bus
            if category not in [1, 2, 3]:
                continue

            if frame_id not in annotations:
                annotations[frame_id] = []

            annotations[frame_id].append((x, y, w, h))

    frame_ids = list(annotations.keys())
    random.shuffle(frame_ids)

    split_index = int(len(frame_ids) * train_ratio)
    train_frames = set(frame_ids[:split_index])

    for frame_id in frame_ids:
        img_name = f"img{frame_id:06d}.jpg"
        img_path = os.path.join(seq_path, img_name)

        if not os.path.exists(img_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue

        height, width = img.shape[:2]

        yolo_lines = []
        for (x, y, bw, bh) in annotations[frame_id]:
            x_center = (x + bw / 2) / width
            y_center = (y + bh / 2) / height
            bw /= width
            bh /= height

            yolo_lines.append(f"0 {x_center} {y_center} {bw} {bh}")

        new_name = f"{seq}_{img_name}"

        if frame_id in train_frames:
            img_dest = os.path.join(train_img_dir, new_name)
            lbl_dest = os.path.join(train_lbl_dir, new_name.replace(".jpg", ".txt"))
        else:
            img_dest = os.path.join(val_img_dir, new_name)
            lbl_dest = os.path.join(val_lbl_dir, new_name.replace(".jpg", ".txt"))

        shutil.copy(img_path, img_dest)

        with open(lbl_dest, "w") as f:
            f.write("\n".join(yolo_lines))

print("✅ UAVDT converted to YOLO format successfully.")
# /Users/harshdeepkalita/datasets/vehicle/
#  ├── images/
#  │    ├── train/
#  │    └── val/
#  ├── labels/
#  │    ├── train/
#  │    └── val/

✅ UAVDT converted to YOLO format successfully.


In [19]:
import os
import shutil

uav_root = "datasets/vehicles"
coco_root = "datasets/coco_vehicle_only"

def merge_split(split_src, split_dst):

    src_img_dir = os.path.join(coco_root, "images", split_src)
    src_lbl_dir = os.path.join(coco_root, "labels", split_src)

    dst_img_dir = os.path.join(uav_root, "images", split_dst)
    dst_lbl_dir = os.path.join(uav_root, "labels", split_dst)

    count = 0

    for label_file in os.listdir(src_lbl_dir):

        src_label_path = os.path.join(src_lbl_dir, label_file)
        img_name = label_file.replace(".txt", ".jpg")
        src_img_path = os.path.join(src_img_dir, img_name)

        if not os.path.exists(src_img_path):
            continue

        # Add prefix to avoid collision
        new_img_name = "coco_" + img_name
        new_lbl_name = "coco_" + label_file

        shutil.copy(src_img_path, os.path.join(dst_img_dir, new_img_name))
        shutil.copy(src_label_path, os.path.join(dst_lbl_dir, new_lbl_name))

        count += 1

    print(f"✅ Merged {count} images into {split_dst}")

# Merge train
merge_split("train2017", "train")

# Merge val
merge_split("val2017", "val")

print("🎯 COCO successfully merged into vehicles dataset.")

✅ Merged 16270 images into train
✅ Merged 707 images into val
🎯 COCO successfully merged into vehicles dataset.


In [ ]:
import torch

print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

model = YOLO("yolo26s.pt")

model.train(
    data="datasets/vehicles/vehicle.yaml",
    imgsz=832,
    epochs=100,
    batch=6,
    device="mps",
    rect=True,
    optimizer="MuSGD",
    cos_lr=True,
    patience=20,
    single_cls=True
)

MPS available: True
MPS built: True
Ultralytics 8.4.14 🚀 Python-3.8.19 torch-2.3.1 MPS (Apple M3 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets/vehicles/vehicle.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=MuSGD, overlap_mask=True, patie